# 실습 : 비슷한 이미지 찾기
openCV기능중 색상히스토그램을 추출해 서로 비교하는 기능을 사용해보려고 한다. 
히스토그램이란 이미지에서 픽셀별 색상값의 분포라고 할수 있다.

![이](https://docs.opencv.org/4.x/histogram_sample.jpg)


위의 예시는 이미지를 흑백으로 변환했을 때 밝기에 따른 히스토그램이지만, 여기에서는 RGB 각 채널별 분포를 사용하여 비교해볼 것이다.
계산량을 감안하여  4개 구간(0\~63, 64\~127, 128\~191, 192\~255)로 나누어 픽셀 수를 나타내보자


# STEP 1. 아이디어 개요

사용자 입력 
- 이미지 경로
모델의 기능
- 검색대상들중 비슷한 이미지를 골라 화면에 표시해줌(5개)
- 비슷한 이미지에 대하여...
 - cv2.compareHist() : 히스토그램의 유사성을 계산해주는 함수 활용

# STEP 2. 아이디어 구체화
유사항 이미지를 선택하는 알고리즘
- 검색대상 이미지를 로드
- 입력이미지와 비교한 유사도를 저장하여 정렬한다.
- 유사도 상위 5개를 고른다

# STEP 3. 핵심 알고리즘 - 유사도 계산 방법

입력 이미지와 비교하여 유사도를 기준으로 순서화하는 방식
- 입력이미지와 검색이미지를 하나씩 히스토그램으로 만듬
- OpenCV의 compareHist() 함수를 사용하여 입력 이미지와 검색 대상 이미지 하나하나의 히스토그램 간 유사도를 계산
- 계산된 유사도를 기준으로 정렬하여 순서화

# STEP 4. 기능의 모듈화
프로그램의 흐름

- 프로그램이 실행된다.
- 입력된 경로의 이미지 파일을 불러온다.
- 검색 대상 이미지들을 불러온다.
- 입력 이미지를 히스토그램으로 만든다.
- 검색 대상 이미지들을 하나하나 히스토그램으로 만든다.
- OpenCV의 compareHist() 함수를 사용하여 입력 이미지와 검색 대상 이미지 하나하나의 히스토그램 간 유사도를 계산한다.
- 계산된 유사도를 기준으로 정렬하여 순서를 매긴다.
- 유사도 순서상으로 상위 5개 이미지를 고른다.
- 고른 이미지들을 표시한다.
- 프로그램이 종료된다.

함수화시킬 기능
- build_histogram_db()
 - 검색 대상 이미지들을 불러온다.( CIFAR-100 이미지로 제한)
 - 검색 대상 이미지들을 하나하나 히스토그램으로 만든다.
- search()
 - OpenCV의 compareHist() 함수를 사용하여 입력 이미지와 검색 대상 이미지 하나하나의 히스토그램 간 유사도를 계산한다.
 - 계산된 유사도를 기준으로 정렬하여 순서를 매긴다.
 - 유사도 순서상으로 상위 5개 이미지를 고른다.

# STEP 5. 설계 과정 완성

- 프로그램이 실행된다.
- build_histogram_db()
 - CIFAR-100 이미지들을 불러온다.
 - CIFAR-100 이미지들을 하나하나 히스토그램으로 만든다.
 - 이미지 이름을 키로 하고, 히스토그램을 값으로 하는 딕셔너리 histogram_db를 반환한다.
- CIFAR-100 히스토그램 중 입력된 이미지 이름에 해당하는 히스토그램을 입력 이미지로 선택하여 target_histogram이라는 변수명으로 지정한다.
- search()
 - 입력 이미지 히스토그램 target_histogram와 전체 검색 대상 이미지들의 히스토그램을 가진 딕셔너리 histogram_db를 입력으로 받는다.
 - OpenCV의 compareHist() 함수를 사용하여 입력 이미지와 검색 대상 이미지 하나하나의 히스토그램 간 유사도를 계산한다. 결과는 result라는 이름의 딕셔너리로, 키는 이미지 이름, 값은 유사도로 한다.
 - 계산된 유사도를 기준으로 정렬하여 순서를 매긴다.
 - 유사도 순서상으로 상위 5개 이미지만 골라서 result에 남긴다.
- 고른 이미지들을 표시한다.
- 프로그램이 종료된다.

# STEP 6. 코드로 구현

함수부터 하나씩 구현해보도록하자

### build_histogram_db()
 - CIFAR-100 이미지들을 불러온다.
 - CIFAR-100 이미지들을 하나하나 히스토그램으로 만든다.
 - 이미지 이름을 키로 하고, 히스토그램을 값으로 하는 딕셔너리 histogram_db를 반환한다.
 - 히스토그램을 만드는 구간은  4개 구간(0\~63, 64\~127, 128\~191, 192\~255)으로 한정한다

In [13]:
import os
import pickle
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

In [6]:
def get_histogram(image):
    histogram = []
    # Create histograms per channels, in 4 bins each.
    for i in range(3): #(0:blue, 1:green, 2:red)
        ch_histogram = cv2.calcHist([image], [i], None, [4], [0,256]) #네번째는 X 축 요소(BIN)의 개수
        histogram.append(ch_histogram)
    
    histogram = np.concatenate(histogram)
    histogram = cv2.normalize(histogram, histogram)
    
    return histogram
    

In [12]:
# get_histogram() 확인용 코드
filename = train[b'filenames'][0].decode()
file_path = os.path.join(images_dir_path, filename)
image = cv2.imread(file_path)
histogram = get_histogram(image)
histogram #첫번째 파일의 histogram (4개구간의 RGB값 총 12개 원소의 배열출력)

array([[0.3126804 ],
       [0.4080744 ],
       [0.14521089],
       [0.21940625],
       [0.18654831],
       [0.23742512],
       [0.30208108],
       [0.35931748],
       [0.06465594],
       [0.35825753],
       [0.36991683],
       [0.29254165]], dtype=float32)

In [14]:
def build_histogram_db():
    histogram_db = {}
    
    #디렉토리에 모아 둔 이미지 파일들을 전부 리스트업합니다. 
    path = image_dir_path
    file_list = os.listdir(path)
    
    #이미지 이름을 키로 하고, 히스토그램을 값으로 하는 딕셔너리 histogram_db 설정
    for filename in file_list:
        file_path = os.path.join(images_dir_path, filename)
        image = cv2.imread(file_path)
        histogram = get_histogram(image) 
        histogram_db[filename] = histogram
        
    return histogram_db

In [ ]:
histogram_db = build_histogram_db()
histogram_db['adriatic_s_001807.png']

전체 이미지에 대해 검색하기위해 시간이 걸림, 이제 histogram_db가 완성되었다. 
target_histogram에 입력받은 이미지의 히스토그램을 저장하는 함수를 만들어 보도록하자


In [ ]:
def get_target_histogram():
    filename = input("이미지 파일명을 입력하세요: ")
    if filename not in histogram_db:
        print('유효하지 않은 이미지 파일명입니다.')
        return None
    return histogram_db[filename]

In [ ]:
target_histogram = get_target_histogram()
target_histogram

### search()
 - 입력 이미지 히스토그램 target_histogram와 전체 검색 대상 이미지들의 히스토그램을 가진 딕셔너리 histogram_db를 입력으로 받는다.
 - OpenCV의 compareHist() 함수를 사용하여 입력 이미지와 검색 대상 이미지 하나하나의 히스토그램 간 유사도를 계산한다. 결과는 result라는 이름의 딕셔너리로, 키는 이미지 이름, 값은 유사도로 한다.
 - 계산된 유사도를 기준으로 정렬하여 순서를 매긴다.
 - 유사도 순서상으로 상위 5개 이미지만 골라서 result에 남긴다.

In [15]:
def search(histogram_db, target_histogram, top_k=5):
    results = {}

    # Calculate similarity distance by comparing histograms.
    for file_name, histogram in tqdm(histogram_db.items()):
        distance = cv2.compareHist(H1=target_histogram,
                                   H2=histogram,
                                   method=cv2.HISTCMP_CHISQR)

        results[file_name] = distance

    results = dict(sorted(results.items(), key=lambda item: item[1])[:top_k])

    return results

In [ ]:
result = search(histogram_db, target_histogram)
result

입력받은 이미지 파일과 가장 유사한 상위 5개의 이미지가 아주 빠르게 검색되어 나왔다. 1번째는 당연히 입력받은 이미지 그 자신이다. 그럼 5개의 이미지가 얼마나 실제로 유사한지 시각화를 해서 알아보자

In [ ]:
def show_result(res):
    f = plt.figure(figsize=(10,3))
    for idx, filename in enumerate(result.keys()):
        img_path = os.path.join(images_dir_path, filename)
        im = f.add_subplot(1,len(result),idx+1)
        img = Image.open(img_path)
        im.imshow(img)
        
    

In [ ]:
show_result(result)

In [ ]:
target_histogram = get_target_histogram()
result = search(histogram_db, target_histogram)
show_result(result)

---

Reference

[Basic Image Manipulations in Python and OpenCV: Resizing (scaling), Rotating, and Cropping - PyImageSearch](https://www.pyimagesearch.com/2014/01/20/basic-image-manipulations-in-python-and-opencv-resizing-scaling-rotating-and-cropping/)

[Clever Girl: A Guide to Utilizing Color Histograms for Computer Vision and Image Search Engines - PyImageSearch](https://www.pyimagesearch.com/2014/01/22/clever-girl-a-guide-to-utilizing-color-histograms-for-computer-vision-and-image-search-engines/)

[Hobbits and Histograms - A How-To Guide to Building Your First Image Search Engine in Python - PyImageSearch](https://www.pyimagesearch.com/2014/01/27/hobbits-and-histograms-a-how-to-guide-to-building-your-first-image-search-engine-in-python/)